# Pi-HEaaN

In [58]:
import piheaan as heaan
from piheaan.math import sort
from piheaan.math import approx # for piheaan math function
import math
import numpy as np
import pandas as pd
import os

In [59]:
# set parameter
params = heaan.ParameterPreset.FGb
context = heaan.make_context(params) # context has paramter information
heaan.make_bootstrappable(context) # make parameter bootstrapable

# create and save keys
key_file_path = "./keys"
sk = heaan.SecretKey(context) # create secret key
os.makedirs(key_file_path, mode=0o775, exist_ok=True)
sk.save(key_file_path+"/secretkey.bin") # save secret key

key_generator = heaan.KeyGenerator(context, sk) # create public key
key_generator.gen_common_keys()
key_generator.save(key_file_path+"/") # save public key

In [60]:
# load secret key and public key
# When a key is created, it can be used again to save a new key without creating a new one
key_file_path = "./keys"

sk = heaan.SecretKey(context,key_file_path+"/secretkey.bin") # load secret key
pk = heaan.KeyPack(context, key_file_path+"/") # load public key
pk.load_enc_key()
pk.load_mult_key()

eval = heaan.HomEvaluator(context,pk) # to load piheaan basic function
dec = heaan.Decryptor(context) # for decrypt
enc = heaan.Encryptor(context) # for encrypt

In [61]:
# log_slots is used for the number of slots per ciphertext
# It depends on the parameter used (ParameterPreset)
# The number '15' is the value for maximum number of slots,
# but you can also use a smaller number (ex. 2, 3, 5, 7 ...)
# The actual number of slots in the ciphertext is calculated as below.
log_slots = 15 
num_slots = 2**log_slots

- The computations in pi-heaan are based on operations between slots.
- Also, the result of an operation between ciphertexts is placed into a new ciphertext (or overwritten).

# 01 heaan.HomEvaluator

### add
- add : addition of two ciphertexts
    - add(self: piheaan.HomEvaluator, arg0: piheaan.Ciphertext, arg1: piheaan.Ciphertext, arg2: piheaan.Ciphertext)
    - add(self: piheaan.HomEvaluator, arg0: piheaan.Ciphertext, arg1: piheaan.Message, arg2: piheaan.Ciphertext)
    - add(self: piheaan.HomEvaluator, arg0: piheaan.Message, arg1: piheaan.Message, arg2: piheaan.Message)
    - arg2 <- arg0 + arg1    

### sub
- sub : subtraction of two ciphertexts
    - same paramters as 'add'

In [62]:
log_slots = 3
num_slots = 2**log_slots

data1 = [0.1, 0.2, 0.3, 4, 0.5, 0.6, 0.7, 8]
message_1 = heaan.Message(log_slots)
for i in range(num_slots):
    message_1[i] = data1[i]

data2 = [0.2, 0.2, 0.3, 0.4, 0.15, 2, 0.1, 8]
message_2 = heaan.Message(log_slots)
for i in range(num_slots):
    message_2[i] = data2[i]


print("message_1 : ", message_1)
print()
print("message_2 : ", message_2)

message_1 :  [ (0.100000+0.000000j), (0.200000+0.000000j), (0.300000+0.000000j), (4.000000+0.000000j), (0.500000+0.000000j), (0.600000+0.000000j), (0.700000+0.000000j), (8.000000+0.000000j) ]

message_2 :  [ (0.200000+0.000000j), (0.200000+0.000000j), (0.300000+0.000000j), (0.400000+0.000000j), (0.150000+0.000000j), (2.000000+0.000000j), (0.100000+0.000000j), (8.000000+0.000000j) ]


In [63]:
# (ciphertext + ciphertext)
ciphertext_1 = heaan.Ciphertext(context)
ciphertext_2 = heaan.Ciphertext(context)
result_add = heaan.Ciphertext(context)

enc.encrypt(message_1, pk, ciphertext_1)
enc.encrypt(message_2, pk, ciphertext_2)

eval.add(ciphertext_1, ciphertext_2, result_add)

result_add_message = heaan.Message(log_slots)
dec.decrypt(result_add, sk, result_add_message)

print("(ciphertext + ciphertext) : ", result_add_message)

(ciphertext + ciphertext) :  [ (0.300000+0.000000j), (0.400000+0.000000j), (0.600000+0.000000j), (4.400000+0.000000j), (0.650000+0.000000j), (2.600000+0.000000j), (0.800000+0.000000j), (16.000000+0.000000j) ]


In [64]:
# (ciphertext + message) 1
eval.add(ciphertext_1, message_2, result_add)

result_add_message = heaan.Message(log_slots)
dec.decrypt(result_add, sk, result_add_message)

print("(ciphertext + message) : ", result_add_message)

(ciphertext + message) :  [ (0.300000+0.000000j), (0.400000+0.000000j), (0.600000+0.000000j), (4.400000+0.000000j), (0.650000+0.000000j), (2.600000+0.000000j), (0.800000+0.000000j), (16.000000+0.000000j) ]


In [65]:
# (ciphertext + message(int or float or ...)) 2
eval.add(ciphertext_1, 2, result_add)

result_add_message = heaan.Message(log_slots)
dec.decrypt(result_add, sk, result_add_message)

print("(ciphertext + message) : ", result_add_message)

(ciphertext + message) :  [ (2.100000+0.000000j), (2.200000+0.000000j), (2.300000+0.000000j), (6.000000+0.000000j), (2.500000+0.000000j), (2.600000+0.000000j), (2.700000+0.000000j), (10.000000+0.000000j) ]


In [66]:
# (message + message)
result_add_message = heaan.Message(log_slots)
eval.add(message_1, message_2, result_add_message)

print("(message + message) : ", result_add_message)

(message + message) :  [ (0.300000+0.000000j), (0.400000+0.000000j), (0.600000+0.000000j), (4.400000+0.000000j), (0.650000+0.000000j), (2.600000+0.000000j), (0.800000+0.000000j), (16.000000+0.000000j) ]


### mult
- multiplication of two ciphertexts
    - mult(self: piheaan.HomEvaluator, arg0: piheaan.Ciphertext, arg1: piheaan.Ciphertext, arg2: piheaan.Ciphertext)
    - mult(self: piheaan.HomEvaluator, arg0: piheaan.Ciphertext, arg1: piheaan.Message, arg2: piheaan.Ciphertext)
    - mult(self: piheaan.HomEvaluator, arg0: piheaan.Message, arg1: piheaan.Message, arg2: piheaan.Message)

In [67]:
# (ciphertext * ciphertext)
result_mult = heaan.Ciphertext(context)
eval.mult(ciphertext_1, ciphertext_2, result_mult)

result_mult_message = heaan.Message(log_slots)
dec.decrypt(result_mult, sk, result_mult_message)

print("(ciphertext * ciphertext) : ", result_mult_message)

(ciphertext * ciphertext) :  [ (0.020000+0.000000j), (0.040000+0.000000j), (0.090000+0.000000j), (1.600000+0.000000j), (0.075000+0.000000j), (1.200000+0.000000j), (0.070000+0.000000j), (64.000000+0.000000j) ]


In [68]:
# (ciphertext * message) 1
eval.mult(ciphertext_1, message_2, result_mult)

result_mult_message = heaan.Message(log_slots)
dec.decrypt(result_mult, sk, result_mult_message)

print("(ciphertext * message) : ", result_mult_message)

(ciphertext * message) :  [ (0.020000+0.000000j), (0.040000+0.000000j), (0.090000+0.000000j), (1.600000+0.000000j), (0.075000+0.000000j), (1.200000+0.000000j), (0.070000+0.000000j), (64.000000+0.000000j) ]


In [69]:
# (ciphertext * message) 2
eval.mult(ciphertext_1, 2, result_mult)

result_mult_message = heaan.Message(log_slots)
dec.decrypt(result_mult, sk, result_mult_message)

print("(ciphertext * ciphertext) : ", result_mult_message)

(ciphertext * ciphertext) :  [ (0.200000+0.000000j), (0.400000+0.000000j), (0.600000+0.000000j), (8.000000+0.000000j), (1.000000+0.000000j), (1.200000+0.000000j), (1.400000+0.000000j), (16.000000+0.000000j) ]


In [70]:
# (message * message)
result_mult_message = heaan.Message(log_slots)
eval.mult(message_1, message_2, result_mult_message)

print("(message * message) : ", result_mult_message)

(message * message) :  [ (0.020000+0.000000j), (0.040000+0.000000j), (0.090000+0.000000j), (1.600000+0.000000j), (0.075000+0.000000j), (1.200000+0.000000j), (0.070000+0.000000j), (64.000000+0.000000j) ]


### negate
- change the sign (ex. -2 -> +2)
- negate(self: piheaan.HomEvaluator, arg0: piheaan.Ciphertext, arg1: piheaan.Ciphertext)

In [71]:
result_negate = heaan.Ciphertext(context)

# plus sign -> minus sign
eval.negate(ciphertext_1, result_negate)

result_negate_message = heaan.Message(log_slots)
dec.decrypt(result_negate, sk, result_negate_message)
print("-(ciphertext_1) : ", result_negate_message)
print()

# minus sign -> plus sign
eval.negate(result_negate, result_negate)

result_negate_message = heaan.Message(log_slots)
dec.decrypt(result_negate, sk, result_negate_message)
print("-(-ciphertext_1) : ", result_negate_message)

-(ciphertext_1) :  [ (-0.100000+0.000000j), (-0.200000+0.000000j), (-0.300000+0.000000j), (-4.000000+0.000000j), (-0.500000+0.000000j), (-0.600000+0.000000j), (-0.700000+0.000000j), (-8.000000+0.000000j) ]

-(-ciphertext_1) :  [ (0.100000+0.000000j), (0.200000+0.000000j), (0.300000+0.000000j), (4.000000+0.000000j), (0.500000+0.000000j), (0.600000+0.000000j), (0.700000+0.000000j), (8.000000+0.000000j) ]


### square
- square of ciphertext (ex. x -> x^2)
- square(self: piheaan.HomEvaluator, arg0: piheaan.Ciphertext, arg1: piheaan.Ciphertext)

In [72]:
result_square = heaan.Ciphertext(context)

eval.square(ciphertext_1, result_square)

result_square_message = heaan.Message(log_slots)
dec.decrypt(result_square, sk, result_square_message)
print("(ciphertext_1)**2 : ", result_square) 
# is the same result as "eval.mult(ciphertext_1, ciphertext_1, result_square)"

(ciphertext_1)**2 :  (level: 23, log(num slots): 3, rescale counter: 0, device: CPU, data: [ (0.010000+0.000000j), (0.040000+0.000000j), (0.090000+0.000000j), (16.000000+0.000000j), (0.250000+0.000000j), (0.360000+0.000000j), (0.490000+0.000000j), (64.000000+0.000000j) ])


### left_rotate / right_rotate
- the operation of rotating data in ciphertext
- left(right)_rotate(self: piheaan.HomEvaluator, arg0: piheaan.Ciphertext, arg1: int, arg2: piheaan.Ciphertext)

In [73]:
result_left_rot = heaan.Ciphertext(context)
result_right_rot = heaan.Ciphertext(context)

# left_rotate
eval.left_rotate(ciphertext_1, 3, result_left_rot)

result_left_rot_message = heaan.Message(log_slots)
dec.decrypt(result_left_rot, sk, result_left_rot_message)
print("left_rotate : ", result_left_rot_message)
print()

# right_rotate
eval.right_rotate(ciphertext_1, 3, result_right_rot)

result_right_rot_message = heaan.Message(log_slots)
dec.decrypt(result_right_rot, sk, result_right_rot_message)
print("right_rotate : ", result_right_rot_message)


left_rotate :  [ (4.000000+0.000000j), (0.500000+0.000000j), (0.600000+0.000000j), (0.700000+0.000000j), (8.000000+0.000000j), (0.100000+0.000000j), (0.200000+0.000000j), (0.300000+0.000000j) ]

right_rotate :  [ (0.600000+0.000000j), (0.700000+0.000000j), (8.000000+0.000000j), (0.100000+0.000000j), (0.200000+0.000000j), (0.300000+0.000000j), (4.000000+0.000000j), (0.500000+0.000000j) ]


### bootstrap / min_level_for_bootstrap
- the level of ciphertext is a value that indicates how many times multiplication can be performed using this ciphertext
- if the level of ciphertext becomes min_level_for_bootstrap, you have to do "bootstrap"
- if the level of ciphertext exceeds 'min_level_for_bootstrap', you cannot perform bootstrap.
- Bootstrap must be done before the level reaches 3 or below
- Then the level is restored
- So you can continue the operation continuously

- bootstrap(self: piheaan.HomEvaluator, arg0: piheaan.Ciphertext, 
		arg1: piheaan.Ciphertext)

In [82]:
# Lets down the level of ciphertext
result_mult = heaan.Ciphertext(context)

eval.mult(ciphertext_1, ciphertext_1, result_mult)
print("1) check ctxt3 level : ", result_mult.level) # if you print 'ciphertext.level', you can check the level of ciphertext
for i in range(result_mult.level - eval.min_level_for_bootstrap):
    eval.mult(result_mult, result_mult, result_mult)
print("2) after 10 times mult.. ctxt3 level : ", result_mult.level) # 23 -> 3
# Now you cannot do multiplication!
# So now..
eval.bootstrap(result_mult, result_mult)
print("3) after bootstrap! ctxt3 level : ", result_mult.level) # now you can do multiplication 12 times

1) check ctxt3 level :  23
2) after 10 times mult.. ctxt3 level :  3
3) after bootstrap! ctxt3 level :  12


In [83]:
# you can check minmum level for bootstrap
eval.min_level_for_bootstrap

3

# 02 heaan.math.sort

### sort
- input range : -0.5 ~ 0.5
- 4th parameter(boolean) : False = descending, True = ascending
- 5th parameter(boolean) is False(default)
- sort(arg0: piheaan.HomEvaluator, 
        arg1: input : piheaan.Ciphertext, 
        arg2: result : piheaan.Ciphertext, 
        arg3: n : int, 
        arg4: ascent : bool, 
        arg5: only_last_stage : bool)

In [110]:
log_slots = 3
num_slots = 2 ** log_slots

data3 = np.random.uniform(size=num_slots)-0.5
message_3 = heaan.Message(log_slots)
for i in range(num_slots):
    message_3[i]=data3[i]
print(message_3)

ciphertext_3 = heaan.Ciphertext(context)
enc.encrypt(message_3, pk, ciphertext_3)

[ (-0.134182+0.000000j), (0.231435+0.000000j), (-0.074949+0.000000j), (-0.252633+0.000000j), (-0.480668+0.000000j), (-0.478114+0.000000j), (0.108921+0.000000j), (-0.012967+0.000000j) ]


In [111]:
# Input range : -0.5 ~ 0.5

ciphertext_out_sort = heaan.Ciphertext(context)
sort.sort(eval, ciphertext_3, ciphertext_out_sort, num_slots, True, False) # 4th boolean: 0: descending, 1 : ascending order

message_out_sort = heaan.Message(log_slots)

dec.decrypt(ciphertext_out_sort, sk, message_out_sort)

print("sort : ", message_out_sort)

index pair in unitSort :0, 1sort :  [ (-0.480668+0.000000j), (-0.478114+0.000000j), (-0.252633+0.000000j), (-0.134182+0.000000j), (-0.074949+0.000000j), (-0.012967+0.000000j), (0.108921+0.000000j), (0.231435+0.000000j) ]

index pair in unitSort :1, 2
index pair in unitSort :1, 1
index pair in unitSort :2, 4
index pair in unitSort :2, 2
index pair in unitSort :2, 1


# 03 heaan.math.approx

### compare
- compare the slots of different ciphertexts and identify which value is greater or smaller
- input range : 2^-18 < |x-y| < 1
- compare(arg0: piheaan.HomEvaluator, 
            arg1: piheaan.Ciphertext, 
            arg2: piheaan.Ciphertext, 
            arg3: piheaan.Ciphertext)

- if the value in slot of arg1 > arg2, then return 1
- if the value in slot of arg1 < arg2, then return 0
- if the value in slot of arg1 == arg2, then return 0.5

In [86]:
data3 = np.random.uniform(size=num_slots)
data4 = np.random.uniform(size=num_slots)
message_3 = heaan.Message(log_slots)
message_4 = heaan.Message(log_slots)
for i in range(num_slots):
    message_3[i]=data3[i]
    message_4[i]=data4[i]

print("message_3 : ", message_3)
print()
print("message_4 : ", message_4)

message_3 :  [ (0.932559+0.000000j), (0.375090+0.000000j), (0.463514+0.000000j), (0.728393+0.000000j), (0.805134+0.000000j), (0.586160+0.000000j), (0.831544+0.000000j), (0.783492+0.000000j) ]

message_4 :  [ (0.130180+0.000000j), (0.387063+0.000000j), (0.738116+0.000000j), (0.779058+0.000000j), (0.784255+0.000000j), (0.070340+0.000000j), (0.356477+0.000000j), (0.407715+0.000000j) ]


In [87]:
ciphertext_3 = heaan.Ciphertext(context)
ciphertext_4 = heaan.Ciphertext(context)
result_compare = heaan.Ciphertext(context)

enc.encrypt(message_3, pk, ciphertext_3)
enc.encrypt(message_4, pk, ciphertext_4)

In [88]:
approx.compare(eval, ciphertext_3, ciphertext_4, result_compare)

result_comp_message = heaan.Message(log_slots)
dec.decrypt(result_compare, sk, result_comp_message)
print("result_comp_message : ", result_comp_message)

result_comp_message :  [ (1.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (1.000000+0.000000j), (1.000000+0.000000j), (1.000000+0.000000j), (1.000000+0.000000j) ]


### discrete_equal
- compare two ciphertexts and identify whether they have the smae value
- input range : |x| ≤ 54 (x : int)
- compare two ciphertexts and identify whether they have the same value
- discrete_equal(eval: piheaan.HomEvaluator, 
                    op1: piheaan.Ciphertext, 
                    op2: piheaan.Ciphertext, 
                    res: piheaan.Ciphertext)


### discrete_equal_zero
- compare a ciphertest with 0 and identify whether the value of the ciphertext is equal to 0
- input range : |x| ≤ 54 (x : int)
- discrete_equal_zero(arg0: piheaan.HomEvaluator, 
                      arg1: piheaan.Ciphertext, 
		              arg2: piheaan.Ciphertext)

In [89]:
data5 = [i for i in range(num_slots)]
data6 = [0, 1, 2, 3, 0, 0, 2, 0]
message_5 = heaan.Message(log_slots)
message_6 = heaan.Message(log_slots)
for i in range(num_slots):
    message_5[i]=data5[i]
    message_6[i]=data6[i]

print("message_5 : ", message_5)
print()
print("message_6 : ", message_6)

message_5 :  [ (0.000000+0.000000j), (1.000000+0.000000j), (2.000000+0.000000j), (3.000000+0.000000j), (4.000000+0.000000j), (5.000000+0.000000j), (6.000000+0.000000j), (7.000000+0.000000j) ]

message_6 :  [ (0.000000+0.000000j), (1.000000+0.000000j), (2.000000+0.000000j), (3.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (2.000000+0.000000j), (0.000000+0.000000j) ]


In [90]:
ciphertext_5 = heaan.Ciphertext(context)
ciphertext_6 = heaan.Ciphertext(context)

enc.encrypt(message_5, pk, ciphertext_5)
enc.encrypt(message_6, pk, ciphertext_6)

# ciphertext5_left_rotate = heaan.Ciphertext(context)  
# eval.left_rotate(ciphertext_5, 5, ciphertext5_left_rotate)  # 5번 암호문을 좌측으로 5칸 로테이션

result_discrete_equal = heaan.Ciphertext(context)
result_discrete_equal_zero = heaan.Ciphertext(context)
approx.discrete_equal(eval, ciphertext_5, ciphertext_6, result_discrete_equal)
approx.discrete_equal_zero(eval, ciphertext_6, result_discrete_equal_zero)
# approx.discrete_equal(eval, ciphertext_5, ciphertext5_left_rotate, result_discrete_equal_rotate)

result_discrete_equal_message = heaan.Message(log_slots)
dec.decrypt(result_discrete_equal, sk, result_discrete_equal_message) # 5번과 6번 메시지 비교

result_discrete_equal_zero_message = heaan.Message(log_slots)
dec.decrypt(result_discrete_equal_zero, sk, result_discrete_equal_zero_message) # 5번과 로테이션 비교

print('message_5 :',message_5)
print()
print('message_6 :', message_6)
print()
print('discrete_equal result : ', result_discrete_equal_message)
print()
print('discrete_equal_zero result : ', result_discrete_equal_zero_message)

message_5 : [ (0.000000+0.000000j), (1.000000+0.000000j), (2.000000+0.000000j), (3.000000+0.000000j), (4.000000+0.000000j), (5.000000+0.000000j), (6.000000+0.000000j), (7.000000+0.000000j) ]

message_6 : [ (0.000000+0.000000j), (1.000000+0.000000j), (2.000000+0.000000j), (3.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (2.000000+0.000000j), (0.000000+0.000000j) ]

discrete_equal result :  [ (1.000000+0.000000j), (1.000000+0.000000j), (1.000000+0.000000j), (1.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j) ]

discrete_equal_zero result :  [ (1.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (1.000000+0.000000j), (1.000000+0.000000j), (0.000000+0.000000j), (1.000000+0.000000j) ]


### inverse
- take the inverse of the value of the ciphertext
- input range : 1 ≤ x ≤ 2^22 or 2^-10 ≤ x ≤ 1
- inverse(arg0: piheaan.HomEvaluator, 
          arg1: piheaan.Ciphertext, 
		  arg2: piheaan.Ciphertext)

          
- inverse(eval: piheaan.HomEvaluator, 
            op: piheaan.Ciphertext, 
            res: piheaan.Ciphertext, 
            init: Optional[float] = None, 
            num_iter: Optional[int] = None, 
            greater_than_one: bool = True)
            
- defaul values are init=pow(2, -18), num_iter=23 if greater_than_one==true,
- otherwise init=pow(2, -10), num_iter=15

In [91]:
data = [i for i in range(1, num_slots+1)]
message = heaan.Message(log_slots)
for i in range(num_slots):
    message[i] = data[i]

In [92]:
ciphertext = heaan.Ciphertext(context)
result_inv = heaan.Ciphertext(context)

enc.encrypt(message, pk, ciphertext)
approx.inverse(eval, ciphertext, result_inv) 

decryptor = heaan.Decryptor(context)
result_inv_message = heaan.Message(log_slots)

decryptor.decrypt(result_inv, sk, result_inv_message)

print('inverse :', result_inv_message) # 

inverse : [ (1.000000+0.000000j), (0.500000+0.000000j), (0.333333+0.000000j), (0.250000+0.000000j), (0.200000+0.000000j), (0.166667+0.000000j), (0.142857+0.000000j), (0.125000+0.000000j) ]


### sigmoid
- sigmoid(arg0: piheaan.HomEvaluator, 
            arg1: piheaan.Ciphertext, 
            arg2: piheaan.Ciphertext, 
            arg3: float)

In [97]:
def sigmoid(x):
    sig = 1 / (1 + math.exp(-x))
    return sig

In [98]:
plaintext_sigmoid = []
for x in message_3:
    plaintext_sigmoid.append(sigmoid(x.real))

print(plaintext_sigmoid)

[0.717594201543754, 0.5926882485214789, 0.613847356909788, 0.6744526371831566, 0.6910716715848441, 0.6424834922627987, 0.6966813858636864, 0.686432314612332]


In [99]:
ciphertext_3 = heaan.Ciphertext(context)
result_sigmoid = heaan.Ciphertext(context)

enc.encrypt(message_3, pk, ciphertext_3)
enc.encrypt(message_4, pk, ciphertext_4)

approx.sigmoid(eval, ciphertext_3, result_sigmoid, 8.0)

result_sigmoid_message = heaan.Message(log_slots)
dec.decrypt(result_sigmoid, sk, result_sigmoid_message)
print("sigmoid : ", result_sigmoid_message)

sigmoid :  [ (0.717596+0.000000j), (0.592686+0.000000j), (0.613846+0.000000j), (0.674453+0.000000j), (0.691073+0.000000j), (0.642483+0.000000j), (0.696683+0.000000j), (0.686433+0.000000j) ]


### sign
- input range 2^-18 < |x| < 1
- sign(arg0: piheaan.HomEvaluator, 
       arg1: piheaan.Ciphertext, 
       arg2: piheaan.Ciphertext, 
       arg3: numiter_g : int, 
       arg4: numiter_f : int)

- default : 3rd paramter = 8 and 4th paramter = 3

In [100]:
data = np.random.uniform(-1, 1, size=num_slots)
message = heaan.Message(log_slots)
for i in range(num_slots):
    message[i] = data[i]
print(message)

ciphertext = heaan.Ciphertext(context)
enc.encrypt(message, pk, ciphertext)

[ (-0.686595+0.000000j), (-0.089386+0.000000j), (-0.851410+0.000000j), (-0.462808+0.000000j), (-0.420806+0.000000j), (0.212859+0.000000j), (-0.516782+0.000000j), (0.596589+0.000000j) ]


In [101]:
result_sign = heaan.Ciphertext(context)

approx.sign(eval, ciphertext, result_sign)

result_sign_message = heaan.Message(log_slots)
dec.decrypt(result_sign, sk, result_sign_message)
print("sign : ", result_sign_message)

sign :  [ (-1.000000+0.000000j), (-1.000000+0.000000j), (-1.000000+0.000000j), (-1.000000+0.000000j), (-1.000000+0.000000j), (1.000000+0.000000j), (-1.000000+0.000000j), (1.000000+0.000000j) ]


### sqrt
- input range : 2^-18 ≤ x ≤ 2
- sqrt(arg0: piheaan.HomEvaluator, 
       arg1: piheaan.Ciphertext, 
       arg2: piheaan.Ciphertext,
       arg3 : init)
       
- The default value for 3rd parameter is 17.
- You can specify a value when you change the default settings.
- As the value of the variable changes, the value of the square root and the value of the ciphertext level change.
- Error when entering 0 or non-integer values



### sqrt_inverse
- x -> 1/sqrt(x)
- input range : 1 ≤ x ≤ 2^22 or 2^-10 ≤ x ≤ 1
- sqrt_inverse(eval: piheaan.HomEvaluator, 
                op: piheaan.Ciphertext, 
                res: piheaan.Ciphertext, 
                init: Optional[float] = None, 
                num_iter: Optional[int] = None, 
                greater_than_one: bool = True)

- default values are init = pow(2, -9), num_iter=20 if greater_than_one==true, 
- otherwise init = pow(2, -5), num_iter=14


In [106]:
log_slots = 3
num_slots = 2 ** log_slots

data = np.random.uniform(2**(-18), 2, num_slots)
# data = [i for i in range(num_slots)]
message = heaan.Message(log_slots)
for i in range(num_slots):
    message[i] = data[i]
print('message :', message)

message : [ (0.505862+0.000000j), (0.489682+0.000000j), (1.861800+0.000000j), (1.346180+0.000000j), (0.865946+0.000000j), (1.349236+0.000000j), (0.590720+0.000000j), (0.204001+0.000000j) ]


In [107]:
ciphertext = heaan.Ciphertext(context)
enc.encrypt(message, pk, ciphertext)

ciphertext_sqrt = heaan.Ciphertext(context)
approx.sqrt(eval, ciphertext, ciphertext_sqrt)

ciphertext_sqrt_inv = heaan.Ciphertext(context)
approx.sqrt_inverse(eval, ciphertext, ciphertext_sqrt_inv) # 1 ~ 2^18

message_out_sqrt = heaan.Message(log_slots)
message_out_sqrt_inv = heaan.Message(log_slots)

In [108]:
dec.decrypt(ciphertext_sqrt, sk, message_out_sqrt)
dec.decrypt(ciphertext_sqrt_inv, sk, message_out_sqrt_inv)

print('sqrt : ', message_out_sqrt)
print()
print('sqrt_inverse : ', message_out_sqrt_inv)

sqrt :  [ (0.711240+0.000000j), (0.699773+0.000000j), (1.364478+0.000000j), (1.160250+0.000000j), (0.930562+0.000000j), (1.161566+0.000000j), (0.768583+0.000000j), (0.451665+0.000000j) ]

sqrt_inverse :  [ (1.405995+0.000000j), (1.429035+0.000000j), (0.732881+0.000000j), (0.861883+0.000000j), (1.074619+0.000000j), (0.860907+0.000000j), (1.301096+0.000000j), (2.213189+0.000000j) ]
